# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
filepath = 'weather_data.csv'
#filepath = 'weather_data_copy.csv'
weather_data = pd.read_csv(filepath)

weather_data

,Unnamed: 0,City,Temperature,%Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Country
0,0,cabo san lucas,32.70,64,18,3.55,22.8909,-109.9124,MX
1,1,airai,10.65,80,76,1.72,-8.9266,125.4092,TL
2,2,upernavik,6.14,69,100,2.57,72.7868,-56.1549,GL
3,3,punta arenas,6.90,81,75,12.86,-53.1500,-70.9167,CL
4,4,qaanaaq,5.12,68,11,1.32,77.4840,-69.3632,GL
...,...,...,...,...,...,...,...,...,...
569,569,guerrero negro,26.23,55,21,6.23,27.9769,-114.0611,MX
570,570,eskil,24.19,22,0,4.44,38.4017,33.4131,TR
571,571,lang son,23.16,92,100,1.30,21.8333,106.7333,VN
572,572,laurel,32.47,63,75,3.60,39.0993,-76.8483,US


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps with API key
gmaps.configure(api_key = g_key)

In [9]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_data[['Latitude', 'Longitude']]
humidity = weather_data['%Humidity']

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.8)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
weather_data


,Unnamed: 0,City,Temperature,%Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Country
0,0,cabo san lucas,32.70,64,18,3.55,22.8909,-109.9124,MX
1,1,airai,10.65,80,76,1.72,-8.9266,125.4092,TL
2,2,upernavik,6.14,69,100,2.57,72.7868,-56.1549,GL
3,3,punta arenas,6.90,81,75,12.86,-53.1500,-70.9167,CL
4,4,qaanaaq,5.12,68,11,1.32,77.4840,-69.3632,GL
...,...,...,...,...,...,...,...,...,...
569,569,guerrero negro,26.23,55,21,6.23,27.9769,-114.0611,MX
570,570,eskil,24.19,22,0,4.44,38.4017,33.4131,TR
571,571,lang son,23.16,92,100,1.30,21.8333,106.7333,VN
572,572,laurel,32.47,63,75,3.60,39.0993,-76.8483,US


In [11]:
#finding weather: <50% Hummidity, <10% Cloudiness, temp 4 - 21 C.
ideal_weather = weather_data.loc[weather_data['%Humidity'] < 50]
ideal_weather = ideal_weather.loc[ideal_weather['Cloudiness'] <= 10]
ideal_weather = ideal_weather.loc[(ideal_weather['Temperature'] >= 4) & (ideal_weather['Temperature'] <= 21)]


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
#creating hotel df
hotel_df = ideal_weather
hotel_df['Hotel Name'] = ''

hotel_df

,Unnamed: 0,City,Temperature,%Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Country,Hotel Name
50,50,graaff-reinet,16.83,38,0,4.03,-32.2522,24.5308,ZA,
106,106,san rafael,17.26,15,0,2.01,-34.6177,-68.3301,AR,
162,162,ovalle,19.51,47,0,1.61,-30.5983,-71.2003,CL,
290,290,chegutu,13.93,30,0,4.11,-18.1302,30.1407,ZW,
330,330,rundu,16.17,29,7,2.80,-17.9333,19.7667,NaN,
353,353,francistown,14.43,44,0,4.74,-21.1700,27.5079,BW,
514,514,trelew,11.79,48,4,9.07,-43.2490,-65.3051,AR,
555,555,comodoro rivadavia,10.92,37,0,15.95,-45.8667,-67.5000,AR,


In [14]:
hotel_list = []
hotel_lat_list = []
hotel_lng_list = []

for index, row in hotel_df.iterrows():
    
    
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params['location'] = f'{lat},{lng}'
    
    response = requests.get(base_url, params=params).json()
    
    #print(response)
    
    try:
        hotel = response['results'][1]['name']
        hotel_lat = response['results'][1]['geometry']['location']['lat']
        hotel_lng = response['results'][1]['geometry']['location']['lng']
                
        hotel_list.append(hotel)
        hotel_lat_list.append(hotel_lat)
        hotel_lng_list.append(hotel_lng)
    
    except:
        print('No hotel found')
    
    

In [15]:
hotel_df['Hotel Name'] = hotel_list

hotel_df

,Unnamed: 0,City,Temperature,%Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Country,Hotel Name
50,50,graaff-reinet,16.83,38,0,4.03,-32.2522,24.5308,ZA,The Whyte House Accommodation
106,106,san rafael,17.26,15,0,2.01,-34.6177,-68.3301,AR,Tower Inn Hotel & Suites Casino Spa Convention...
162,162,ovalle,19.51,47,0,1.61,-30.5983,-71.2003,CL,Hotel Keo
290,290,chegutu,13.93,30,0,4.11,-18.1302,30.1407,ZW,Jacaranda Lodge
330,330,rundu,16.17,29,7,2.80,-17.9333,19.7667,NaN,"Urban Farm Haus, Rundu, Namibia"
353,353,francistown,14.43,44,0,4.74,-21.1700,27.5079,BW,Cresta Marang Gardens
514,514,trelew,11.79,48,4,9.07,-43.2490,-65.3051,AR,Rayentray Trelew
555,555,comodoro rivadavia,10.92,37,0,15.95,-45.8667,-67.5000,AR,Comodoro Hotel


In [16]:
# NOTE: Do not change any of the code in this cell

# deleted line: <dt>Country</dt><dd>{Country}</dd>

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [17]:
countries = hotel_df["Country"].tolist()
hotel_names = hotel_df["Hotel Name"].tolist()
cities = hotel_df['City'].tolist()

In [21]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    locations, info_box_content= hotel_info)

fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))